In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.preprocessing import scale
from sklearn.preprocessing import LabelBinarizer

In [2]:
Train_Data = pd.read_csv("datasets/Train.csv")
Train_Data

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results,IsUnderRisk
0,2,8.032,14,8,3,6,0,1
1,31,77.730,8,3,3,8,1,0
2,40,59.203,3,12,11,3,0,1
3,12,73.080,4,5,7,6,0,0
4,4,15.666,13,15,6,7,2,1
...,...,...,...,...,...,...,...,...
538,16,74.017,7,4,5,7,1,0
539,2,70.460,7,5,6,4,0,0
540,1,79.243,7,5,3,8,1,0
541,40,69.140,7,8,4,5,1,1


In [3]:
Test_Data = pd.read_csv("datasets/Test.csv")
Test_Data

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results
0,41,18.272,13,12,9,7,0
1,17,64.799,6,10,7,4,1
2,31,68.890,3,3,7,8,0
3,3,16.492,15,10,7,4,1
4,9,17.178,7,3,8,3,1
...,...,...,...,...,...,...,...
228,2,73.165,6,5,5,5,0
229,2,6.952,8,8,5,4,1
230,3,6.796,7,5,4,5,1
231,3,9.197,13,12,10,7,1


In [4]:
Train_Data.describe()

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results,IsUnderRisk
count,543.000000,543.000000,543.000000,543.000000,543.000000,543.000000,543.000000,543.000000
mean,19.576427,32.259346,8.189687,7.327808,7.046041,5.530387,0.613260,0.626151
std,14.722687,24.887291,3.312022,3.450667,3.156978,1.839124,0.764509,0.484270
min,0.000000,5.185000,3.000000,3.000000,3.000000,3.000000,0.000000,0.000000
25%,6.000000,13.565000,6.000000,5.000000,5.000000,4.000000,0.000000,0.000000
50%,14.000000,19.803000,8.000000,7.000000,7.000000,6.000000,1.000000,1.000000
75%,37.000000,62.594000,11.000000,10.000000,8.000000,7.000000,1.000000,1.000000
max,44.000000,80.809000,15.000000,15.000000,15.000000,13.000000,10.000000,1.000000


In [5]:
Test_Data.describe()

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results
count,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000,233.000000
mean,19.343348,32.591034,7.854077,7.424893,6.957082,5.648069,0.579399
std,14.915933,25.185948,3.176584,3.525117,3.337077,1.708424,0.703382
min,0.000000,5.469000,3.000000,3.000000,3.000000,3.000000,0.000000
25%,6.000000,12.156000,6.000000,5.000000,4.000000,4.000000,0.000000
50%,13.000000,19.968000,7.000000,7.000000,6.000000,6.000000,1.000000
75%,37.000000,62.766000,10.000000,10.000000,8.000000,7.000000,1.000000
max,41.000000,77.668000,15.000000,15.000000,15.000000,10.000000,5.000000


In [6]:
# Train_Data.isnull().sum()

In [7]:
# Test_Data.isnull().sum()

In [8]:
Train_Data['IsUnderRisk'].value_counts()

1    340
0    203
Name: IsUnderRisk, dtype: int64

In [9]:
Train_Data['Past_Results'].value_counts()

1     272
0     250
2      13
3       5
10      1
6       1
4       1
Name: Past_Results, dtype: int64

In [10]:
def combineData():
    train = pd.read_csv("datasets/Train.csv")
    test = pd.read_csv("datasets/Test.csv")
    target = train.IsUnderRisk
    train.drop('IsUnderRisk', 1, inplace=True)
    combined = train.append(test)
    combined.reset_index(inplace=True)
    combined.drop(['index'], inplace=True, axis=1)
    return combined

In [11]:
CombinedData = combineData()
CombinedData

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results
0,2,8.032,14,8,3,6,0
1,31,77.730,8,3,3,8,1
2,40,59.203,3,12,11,3,0
3,12,73.080,4,5,7,6,0
4,4,15.666,13,15,6,7,2
...,...,...,...,...,...,...,...
771,2,73.165,6,5,5,5,0
772,2,6.952,8,8,5,4,1
773,3,6.796,7,5,4,5,1
774,3,9.197,13,12,10,7,1


In [12]:
CombinedData.describe()

,City,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,19.506443,32.358938,8.088918,7.356959,7.019330,5.565722,0.603093
std,14.771724,24.961530,3.273523,3.471194,3.210208,1.800591,0.746391
min,0.000000,5.185000,3.000000,3.000000,3.000000,3.000000,0.000000
25%,6.000000,13.044250,6.000000,5.000000,4.000000,4.000000,0.000000
50%,13.000000,19.844000,8.000000,7.000000,6.000000,6.000000,1.000000
75%,37.000000,62.680750,11.000000,10.000000,8.000000,7.000000,1.000000
max,44.000000,80.809000,15.000000,15.000000,15.000000,13.000000,10.000000


In [13]:
CombinedData['City'].unique()#value_counts()

array([ 2, 31, 40, 12,  4,  1,  9, 23, 38,  0, 10,  8, 11, 19,  6, 37,  3,
        5, 41, 16, 22, 29, 28, 18, 35, 13, 32, 14, 30, 43, 39, 21, 24, 25,
       17, 33, 15, 27, 20,  7, 42, 44, 26, 34, 36], dtype=int64)

# Label Encode City

In [14]:
def labelEncode(df):
    df = df.astype('category')
    df2 = df.cat.codes
    return df2

# CombinedData['City'] = labelEncode(CombinedData['City'])
CombinedData.drop('City', 1, inplace=True)

# Now Scaling the features

In [15]:
def featureScale(df):
    df /= df.max()
    return df
    

In [16]:
CombinedData['Location_Score'] /= 100 
CombinedData['Internal_Audit_Score'] = featureScale(CombinedData['Internal_Audit_Score'])
CombinedData['External_Audit_Score'] = featureScale(CombinedData['External_Audit_Score'])
CombinedData['Fin_Score'] = featureScale(CombinedData['Fin_Score'])
CombinedData['Loss_score'] = featureScale(CombinedData['Loss_score'])
CombinedData['Past_Results'] = featureScale(CombinedData['Past_Results'])

In [17]:
CombinedData.describe()

,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,0.323589,0.539261,0.490464,0.467955,0.428132,0.060309
std,0.249615,0.218235,0.231413,0.214014,0.138507,0.074639
min,0.051850,0.200000,0.200000,0.200000,0.230769,0.000000
25%,0.130443,0.400000,0.333333,0.266667,0.307692,0.000000
50%,0.198440,0.533333,0.466667,0.400000,0.461538,0.100000
75%,0.626807,0.733333,0.666667,0.533333,0.538462,0.100000
max,0.808090,1.000000,1.000000,1.000000,1.000000,1.000000


# Making New Features

In [18]:
CombinedData['Total_Audit_Score'] = (CombinedData['Internal_Audit_Score'] + CombinedData['External_Audit_Score'])/2

In [19]:
CombinedData.describe()

,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results,Total_Audit_Score
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,0.323589,0.539261,0.490464,0.467955,0.428132,0.060309,0.514863
std,0.249615,0.218235,0.231413,0.214014,0.138507,0.074639,0.189841
min,0.051850,0.200000,0.200000,0.200000,0.230769,0.000000,0.200000
25%,0.130443,0.400000,0.333333,0.266667,0.307692,0.000000,0.366667
50%,0.198440,0.533333,0.466667,0.400000,0.461538,0.100000,0.466667
75%,0.626807,0.733333,0.666667,0.533333,0.538462,0.100000,0.666667
max,0.808090,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
CombinedData['Internal/Total'] = CombinedData['Internal_Audit_Score']/CombinedData['Total_Audit_Score']
CombinedData['External/Total'] = CombinedData['External_Audit_Score']/CombinedData['Total_Audit_Score']

In [21]:
def recover_train_test_target():
    global CombinedData, Train_Data
    targets = Train_Data['IsUnderRisk']
    train = CombinedData.head(543)
    test = CombinedData.iloc[543:]
    return train, targets, test

In [22]:
X,y,Test = recover_train_test_target()

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [24]:
y_pred = logreg.predict(X_test)

In [25]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.79


In [26]:
pd.crosstab(y_test, y_pred, rownames=['Actual Result'], colnames=['Predicted Result'])

Predicted Result,0,1
Actual Result,,
0,31,12
1,11,55


In [27]:
# features = pd.DataFrame()
# features['Feature'] = X.columns
# features['Importance'] = logreg.
# features.sort_values(by=['Importance'], ascending=False, inplace=True)
# features.set_index('Feature', inplace=True)

In [28]:
from sklearn.metrics import classification_report

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.72      0.73        43
           1       0.82      0.83      0.83        66

    accuracy                           0.79       109
   macro avg       0.78      0.78      0.78       109
weighted avg       0.79      0.79      0.79       109



In [30]:
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier

In [31]:
#model = LGBMClassifier()
model = XGBClassifier()
# model = neighbors.KNeighborsClassifier()
# model = RandomForestClassifier(n_estimators=50, max_features='sqrt')
# model = CatBoostClassifier(iterations=1000)
model.fit(X_train, y_train)

[01:09:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\python38\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [32]:
accuracy = model.score(X_test,y_test)
accuracy

0.7889908256880734

In [33]:
y_pred = model.predict(X_test)
y_pred

array([1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1],
      dtype=int64)

In [34]:
from sklearn.metrics import log_loss

In [35]:
log_loss(y_test, y_pred)

7.288086717909249

In [36]:
test_result = model.predict_proba(Test)

In [37]:
result = pd.DataFrame(test_result)

In [38]:
result

,0,1
0,0.000082,0.999918
1,0.045806,0.954194
2,0.857633,0.142367
3,0.001492,0.998508
4,0.050970,0.949030
...,...,...
228,0.983603,0.016397
229,0.031296,0.968704
230,0.876812,0.123188
231,0.002148,0.997852


In [39]:
result.to_csv("result.csv",index=False)

In [40]:
CombinedData

,Location_Score,Internal_Audit_Score,External_Audit_Score,Fin_Score,Loss_score,Past_Results,Total_Audit_Score,Internal/Total,External/Total
0,0.08032,0.933333,0.533333,0.200000,0.461538,0.0,0.733333,1.272727,0.727273
1,0.77730,0.533333,0.200000,0.200000,0.615385,0.1,0.366667,1.454545,0.545455
2,0.59203,0.200000,0.800000,0.733333,0.230769,0.0,0.500000,0.400000,1.600000
3,0.73080,0.266667,0.333333,0.466667,0.461538,0.0,0.300000,0.888889,1.111111
4,0.15666,0.866667,1.000000,0.400000,0.538462,0.2,0.933333,0.928571,1.071429
...,...,...,...,...,...,...,...,...,...
771,0.73165,0.400000,0.333333,0.333333,0.384615,0.0,0.366667,1.090909,0.909091
772,0.06952,0.533333,0.533333,0.333333,0.307692,0.1,0.533333,1.000000,1.000000
773,0.06796,0.466667,0.333333,0.266667,0.384615,0.1,0.400000,1.166667,0.833333
774,0.09197,0.866667,0.800000,0.666667,0.538462,0.1,0.833333,1.040000,0.960000


In [41]:
import pickle

In [42]:
# Creating a pickle file for the classifier
filename = 'model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [43]:
# np.array(X_test)

In [44]:
inp = np.array([[0.07,0.67,0.73,0.93,0.30,0.1,0.7, 0.95,1.047]])

In [49]:
op = logreg.predict(inp)
op

array([1], dtype=int64)

In [52]:
test_result2 = logreg.predict_proba(inp)

In [53]:
result2 = pd.DataFrame(test_result)
result2

,0,1
0,0.000082,0.999918
1,0.045806,0.954194
2,0.857633,0.142367
3,0.001492,0.998508
4,0.050970,0.949030
...,...,...
228,0.983603,0.016397
229,0.031296,0.968704
230,0.876812,0.123188
231,0.002148,0.997852


In [54]:
filename1 = 'model2.pkl'
pickle.dump(model, open(filename1, 'wb'))

In [47]:
# model.get_booster().feature_names

['Location_Score',
 'Internal_Audit_Score',
 'External_Audit_Score',
 'Fin_Score',
 'Loss_score',
 'Past_Results',
 'Total_Audit_Score',
 'Internal/Total',
 'External/Total']

In [55]:
# X_train